<a href="https://colab.research.google.com/github/usnhee/TIL-/blob/master/tensor_basic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense( 32,activation='relu', input_shape=(784,)))
model.add(layers.Dense(10, activation='softmax'))



In [ ]:
from keras import models
from keras import layers
input_tensor = layers.Input(shape=(784,))
x = layers.Dense(32, activation='relu')(input_tensor)
output_tensor = layers.Dense(10, activation= 'softmax')(x)

model = models.Model(inputs=input_tensor, outputs=output_tensor)

## Keras 
- 모델 구조가 정의된 후에는 Sequential 모델을 사용했는지 함수형 API를 사용했는지 상관 없음. 
- 컴파일 단계에서 학습관정이 설정 -> 모델이 사용할 옵티마이저와 손실함수, 훈련하는 동안 모니터링하기 위해 필요한 측정지표를 지정 


In [22]:
from keras import optimizers 
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='mse', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
model.fit(input_tensor, target_tensor, batch_size=128, epochs=10)

### 영화 리뷰 분류- 이진분류
- Positive / Negative 로 분류 
- IMDB 데이터셋 (전처리 되어있음. 케라스)
- 훈련데이터 25,000개, 테스트데이터 25,000개 
- positive: negative (50:50)
-  각 리뷰(단어 시퀀스)가 숫자 시퀀스로 변환되어 있음. -> 각 숫자는 사전에 있는 고유한 단어 
- num_words=10000 매개변수는 훈련데이터에서 가장 자주 나타나는 단어 1만개만 사용하겠다는 의미

In [3]:
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)


17473536/17464789 [==============================] - 0s 0us/step


In [4]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
print(train_labels[0])

1


In [6]:
print(max([max(sequence) for sequence in train_data]))

9999


In [7]:
word_index = imdb.get_word_index()
reverse_word_index = dict([value,key] for (key, value) in word_index.items())
decoded_review = ' '.join([reverse_word_index.get(i-3, '?') for i in train_data[0]])
print(decoded_review)

1654784/1641221 [==============================] - 0s 0us/step
? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amaz

- 신경망에 숫자리스트를 주입 불가 -> 리스트를 텐서로 만들기 
  - 같은 길이가 되도록 리스트에 패딩(padding)을 추가하고 (samples, sequence_length) 크기의 정수 텐서로 변환 -> 정수 텐서를 다룰 수 있는 층(embedding)을 신경망의 첫번째 층으로 사용 
  - 리스트를 원-핫 인코딩(one-hot encoding) 하여 0과 1의 벡터로 변환 -> 부동 소수 벡터데이터를 다룰 수 있는 Dense층을 신경망의 첫번째 층으로 사용 

In [10]:
import numpy as np 

def vectorize_sequences(seqs, dim=10000):
  results = np.zeros((len(seqs), dim))
  for i, seq in enumerate(seqs): 
    results[i,seq] = 1    # 특정 인덱스 위치를 1로 만듬 
  return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

print(x_train[0])

[0. 1. 1. ... 0. 0. 0.]


In [11]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

## 신경망 모델 만들기 
- 입력데이터가 벡터, 레이블은 스칼라(0 or 1)
- 이런 문제에 잘 작동하는 네트워크 종류는 relu 활성화함수 사용한 완전연결층 Dense(16,activation='relu') 을 그냥 쌓은것. 
- 매개변수 16은 은닉유닛(hidden unit)의 갯수 -> 가중치 행렬 W의 크기가 (input_dimension, 16) -> 입력데이터와 W를 점곱하면 입력데이터가 16차원으로 표현된 공간으로 투영 
- 표현공간의 차원 -> 신경망이 내재된 표현을 학습할 때 가질 수 있는 자유도

- 은닉유닛을 늘리면(표현공간을 더 고차원으로) 신경망이 더 복잡한 표현 학습 가능.-> 계산비용이 커지고 원치않는 패턴을 학습할 수도 있음.
(훈련데이터에서는 성능향상되나, 테스트 데이터에서는 그렇지 않은 패턴)
-Dense층을 쌓을 때 두가지 중요한 구조상 결정 필요
  - how many layers? 
  - how much hidden unit for each layer? 

In [12]:
model = models.Sequential()
model.add(layers.Dense(16, activation = 'relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

### 손실함수와 옵티마이저 선택 
- 이진분류 문제 + 신경말 출력(->확률) : 마지막에 시그모이드 활성화함수를 사용한 하나의 유닛으로 된 층 사용 
- binary crossentropy 손실함수가 적합 
- MSE도 사용가능 
- 확률을 출력하는 모델: 크로스엔트로피가 최선의 선택 

-rmsprop 옵티마이저와 binary_crossentropy 손실함수 모델을 설정 
-훈련하는 동안 accuracy 사용해 모니터링.

In [24]:
from keras import optimizers
from tensorflow.keras import optimizers
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [27]:
from keras import losses
from keras import metrics
from tensorflow.keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss=losses.binary_crossentropy, metrics=[metrics.binary_accuracy])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


### train validation (훈련 검증)
- 훈련하는 동안 처음 본 데이터에 대한 모델의 정확도 측정
- 원본 훈련 데이터에서 10000개의 샘플을 떼어 검증셋을 만듬 

In [28]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[:10000]

- 모델을 512개의 샘플씩 미니 batch를 만들어 20번의 epoch 동안 훈련 
(x_train과 t_train 텐서에 있는 모든 샘플에 대해 20번 반복) 
- 동시에 따로 떼어놓은 1만개 샘플에서 손실과 정확도 측정

In [29]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(partial_x_train, partial_y_train, epochs=50, batch_size=512, validation_data=(x_val,y_val))

ValueError: ignored

In [ ]:
history_dict = history.history
history_dict.keys()

In [30]:
import matplotlib.pyplot as plt
history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss)+1)

plt.figure(figsize=(16,12))
plt.plot(epochs, loss, 'bo', label ='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

NameError: ignored

In [ ]:
plt.clf()
acc = history_dict['acc']
val_acc = history_dict['val_acc']

plt.figure(figsize=(16,12))
plt.plot(epochs, acc, 'bo', label ='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

- 훈련손실이 에포크마다 감소, 정확도는 에포크마다 증가 
- 경사하강법 최적화를 사용했을 때 반복마다 최소화되는 것이 손실이므로 기대와 같다. 
- 검증손실과 검증정확도는 훈련손실/훈련정확도와 다르다. 
- 4번째 에포크에서 역전 

- 훈련셋에서 잘 작동하는 모델이 처음보는 데이터에서 자 작동하지 않음 -> Overfitting 
- 훈련데이터에 대한 성능 향상될수록 신경망이 overfitted -> 항상 훈련셋 이외 데이터에서 성능을 모니터링 해야함. 

In [ ]:
# overfitting 방지를 위해 세번째 에포크 이후 훈련 중지 
model = models.Sequential()   # 은닉층
model.add(layers.Dense(16,activation='relu', input_shape=(10000,)))   #은닉층
model.add(layers.Dense(16,activation='relu'))    #은닉층
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=4, batch_size=512)
results= model.evaluate(x_test, y_test)
print(results)

In [ ]:
# 훈련된 모델로 새로운 데이터에 대해 예측
model.predict(x_test)